# Items related to generating an Arc Model

In [124]:
# imports
import json
from astropy.table import Table, vstack
import pypit

from pypit import pyputils
from pypit import arutils
msgs = pyputils.get_dummy_logger()
from pypit import ararcmodel as pamodel

## Line tables

### Begin with LRISr, R600

In [3]:
# Load
lrisr_file = os.getenv('DROPBOX_DIR')+'PYPIT/Holy_Grail/holy_grail/inputs/lrisr_600_7500_holy.json'
with open(lrisr_file,'r') as f:
    lrisr = json.load(f)

In [106]:
npix = len(lrisr['spec'])

### Full Table

In [118]:
lrisr_tbl = Table()

In [119]:
lrisr_tbl['ion'] = lrisr['ions']
lrisr_tbl['wave'] = lrisr['yfit']
lrisr_tbl['NIST'] = 1
lrisr_tbl['Origin'] = 'LRISr             '

In [120]:
lrisr_tbl[0:5]

ion,wave,NIST,Origin
unicode3,float64,int64,str18
HgI,5771.21,1,LRISr
HgI,5792.276,1,LRISr
NeI,5854.1101,1,LRISr
KrI,5872.5432,1,LRISr
NeI,5883.5252,1,LRISr


### Extras

In [111]:
wave = arutils.func_val(lrisr['fitc'], np.array(lrisr['tcent'])/(npix-1), lrisr['function'], minv=lrisr['fmin'], maxv=lrisr['fmax'])
wave[0:5]

array([ 5771.26248628,  5792.27178383,  5854.15103369,  5872.56940052,
        5883.51000008])

In [114]:
extras = []
for iwave in wave:
    if np.min(np.abs(lrisr_tbl['wave'].data-iwave)) > 0.5:
        extras.append(iwave)
extras[0:5]

[6129.8094923151184,
 6329.5403134529697,
 6364.155954942953,
 6422.0313714678696,
 6440.3180037727343]

In [123]:
extra_tbl = Table()
extra_tbl['wave'] = extras
extra_tbl['ion'] = 'UNK'
extra_tbl['NIST'] = 0
extra_tbl['Origin'] = 'LRISr_ArHgKrNeXe'

In [129]:
tot_tbl = vstack([lrisr_tbl,extra_tbl])
tot_tbl.sort('wave')

### Amplitude

In [31]:
spec = np.array(lrisr['spec'])
npix = spec.size
#
amps = []
for jj,xfit in enumerate(lrisr['xfit']):
    pix = int(np.round(xfit*(npix-1)))
    amps.append(spec[pix])

In [32]:
lrisr_tbl['amp'] = amps

### Generate Individual files

In [44]:
reload(pamodel)
pamodel.add_to_lists(lrisr_tbl)

[INFO]    :: Generate new arc model table: /Users/xavier/local/Python/PYPIT/pypit/data/arc_lines/Model/ArI_model.ascii
[INFO]    :: Generate new arc model table: /Users/xavier/local/Python/PYPIT/pypit/data/arc_lines/Model/HgI_model.ascii
[INFO]    :: Generate new arc model table: /Users/xavier/local/Python/PYPIT/pypit/data/arc_lines/Model/KrI_model.ascii
[INFO]    :: Generate new arc model table: /Users/xavier/local/Python/PYPIT/pypit/data/arc_lines/Model/NeI_model.ascii
[INFO]    :: Generate new arc model table: /Users/xavier/local/Python/PYPIT/pypit/data/arc_lines/Model/XeI_model.ascii


## Match to list

### Simple tests first

In [56]:
lrisr_tbl[9]

ion,wave,NIST,Origin,amp
unicode3,float64,int64,str11,float64
NeI,6097.8506,1,LRISr,14599.2635971


In [85]:
## Simple test
idx = 32
spec_lines = np.array(lrisr['xfit'][idx:idx+4]) * (npix-1)
true_wv = lrisr['yfit'][idx:idx+4]
#
wvmnx = [true_wv[0]-100., true_wv[0]+100]
#
spec_lines, true_wv

(array([ 1186.30899551,  1217.14719081,  1226.39690322,  1233.1139731 ]),
 [7440.9469, 7490.9335, 7505.935, 7516.721])

In [86]:
reload(pamodel)
matches = pamodel.match_quad_to_list(spec_lines, lrisr_tbl['wave'].data, wvmnx, 1.6)

Trying 7386.01
Trying 7440.95
Trying 7490.93
Trying 7505.94
Trying 7516.72
Trying 7537.85


In [87]:
matches

[[32, 33, 34, 35]]

### Loop through all

In [88]:
nlin = len(lrisr['xfit'])

In [133]:
# Init
reload(pamodel)
swv_uncertainty = 200.
#
final_idx = {}
for ii in range(nlin):
    final_idx[ii] = []

In [134]:
# Loop me
#wvdata = lrisr_tbl['wave'].data
wvdata = tot_tbl['wave'].data
for idx in range(nlin-4):
    # Setup
    spec_lines = np.array(lrisr['xfit'][idx:idx+4]) * (npix-1)
    wvmnx = [lrisr['yfit'][idx]-swv_uncertainty, lrisr['yfit'][idx]+swv_uncertainty]
    # Run
    matches = pamodel.match_quad_to_list(spec_lines, wvdata, wvmnx, 1.6, tol=1)
    # Save
    for match in matches:
        for ii in range(4):
            final_idx[idx+ii].append(match[ii])

In [135]:
final_idx

{0: [0],
 1: [1, 1],
 2: [2, 2, 2],
 3: [3, 3, 3, 3],
 4: [4, 4, 4, 4],
 5: [5, 5, 5, 5],
 6: [6, 6, 6, 6],
 7: [7, 7, 7, 7],
 8: [8, 8, 8, 8],
 9: [9, 9, 9, 9],
 10: [11, 11, 11, 11],
 11: [12, 12, 12, 12],
 12: [13, 13, 13, 13],
 13: [14, 14, 14, 14],
 14: [15, 15, 15, 15],
 15: [17, 17, 17, 15, 17, 18],
 16: [19, 19, 18, 19, 20, 19],
 17: [20, 19, 20, 21, 20, 13, 20],
 18: [25, 25, 25, 25, 15, 25, 25],
 19: [26, 16, 26, 26, 26],
 20: [19, 27, 27, 27, 27],
 21: [28, 28, 28, 28],
 22: [29, 29, 29, 29],
 23: [31, 31, 31, 31],
 24: [32, 32, 32, 32],
 25: [34, 34, 34, 34],
 26: [36, 36, 36, 36],
 27: [37, 37, 37, 37],
 28: [38, 38, 38, 38],
 29: [40, 40, 40, 40, 43],
 30: [41, 41, 41, 44, 41, 44],
 31: [42, 42, 50, 42, 50, 42],
 32: [43, 52, 43, 52, 43, 43],
 33: [45, 55, 45, 45, 45],
 34: [46, 46, 46, 46],
 35: [47, 47, 47, 47],
 36: [48, 48, 48, 48],
 37: [49, 49, 49, 49],
 38: [50, 50, 50, 50],
 39: [54, 54, 54, 54],
 40: [56, 56, 56, 56],
 41: [58, 58, 58, 58],
 42: [60, 60, 60, 60],

### Now with other lines in linelist

In [102]:
len(lrisr['tcent']), len(lrisr['xfit'])

(92, 53)